In [1]:
import pandas as pd
import numpy as np
import argparse
import os
import json
import time
import pickle

import trinity.spec as S
from trinity.enumerator import LineSkeletonEnumerator
from trinity.interpreter import MorpheusInterpreter, MorpheusCoarseAbstractInterpreter, MorpheusPartialInterpreter, MorpheusFineAbstractInterpreter
from trinity.decider import Example, MorpheusDecider
from trinity.synthesizer import MorpheusSynthesizer
from trinity.logger import get_logger
from trinity.recorder import get_recorder, set_recorder_dest
from trinity.profiler import get_profiler_pool

In [2]:
from test_morpheus_benchmark import load_example

In [3]:
logger = get_logger('trinity')
set_recorder_dest(os.path.abspath("./logs"))
args = {
    "dsl": "test_min",
    "benchmark": 5,
    "skeletons": "test_min",
    "record": False,
}

In [4]:
# load dsl spec
spec = None
if args["dsl"].startswith("test"):
    spec = S.parse_file("./benchmarks/morpheus/{}/{}.dsl".format(args["benchmark"], args["dsl"]))
else:
    raise Exception("You should not reach here.")

# load benchmark
print("# loading benchmark...")
benchmark_example = load_example(args["benchmark"])

print("# input0 type: {}".format(benchmark_example.input[0].dtypes.tolist()))
print("# input0 is: {}".format(benchmark_example.input[0]))
print("# output type: {}".format(benchmark_example.output.dtypes.tolist()))
print("# output is: {}".format(benchmark_example.output))

# loading benchmark...
# input0 type: [dtype('int64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64')]
# input0 is:    ID     T   P.1  P.2  Q.1
0   1  24.3  10.2  5.5  4.5
1   2  23.4  10.4  5.7  3.2
# output type: [dtype('int64'), dtype('O'), dtype('float64'), dtype('float64')]
# output is:    ID Channel     T     P
0   1       1  24.3  10.2
1   2       1  23.4  10.4
2   1       2  24.3   5.5
3   2       2  23.4   5.7
4   1       1  24.3   4.5
5   2       1  23.4   3.2


In [5]:
# wrap priority pools
# select(separate(gather(@param0, ['-99', '-1']), 2), ['-3'])
priority_pools = [
    # only 1 skeleton, order list according to depth-first post-order order
    [
        # arguments for gather
        [
            spec.get_enum_production(spec.get_type("ColList"),['-99', '-1'])
        ],
        # arguments for separate
        [
            spec.get_enum_production(spec.get_type("ColInt"),"2")
        ],
        # arguments for select
        [
            spec.get_enum_production(spec.get_type("ColList"),['-3'])
        ]
    ]
]

In [6]:
# load skeleton
print("# loading skeleton list...")
skeleton_list = None
if args["skeletons"].startswith("test"):
    with open("./benchmarks/morpheus/{}/{}_skeletons.json".format(args["benchmark"], args["skeletons"]), "r") as f:
        skeleton_list = json.load(f)
else:
    with open("./skeletons/{}_skeletons.json".format(args["skeletons"]), "r") as f:
        skeleton_list = json.load(f)

enumerator = LineSkeletonEnumerator( spec=spec, cands=skeleton_list, pools=priority_pools )
interpreter = MorpheusInterpreter( spec=spec )
# coarse: perform skeleton level abstract interpretation, throws SkeletonAssertion
coarse_abstract_interpreter = MorpheusCoarseAbstractInterpreter()
# fine: perform finer level (sub-skeleton level) abstract interpretation throws EnumAssertion
fine_abstract_interpreter = MorpheusFineAbstractInterpreter( spec=spec )
partial_interpreter = MorpheusPartialInterpreter(
    interpreter=interpreter,
    # partial interpreter currently only works with coarse abstract interpreter
    # abstract_interpreter=fine_abstract_interpreter,
    abstract_interpreter=coarse_abstract_interpreter,
)

# print some basic info
# print("# search space (skeletons): {}".format(len(enumerator._iters)))
# print("# search space (concrete programs): {}".format(
#     sum([len(p) for p in enumerator._iters])
# ))
# input("press to start")
recorder = None
if args["record"]:
    recorder = get_recorder("test.b={}.d={}.s={}".format(
        args["benchmark"], args["dsl"], args["skeletons"]
    ))

decider = MorpheusDecider( 
    interpreter=interpreter, 
    coarse_abstract_interpreter=coarse_abstract_interpreter,
    # fine_abstract_interpreter=fine_abstract_interpreter,
    partial_interpreter=partial_interpreter,
    examples=[benchmark_example], 
    equal_output=interpreter.equal_tb,
    recorder=recorder,
)
synthesizer = MorpheusSynthesizer(
    enumerator=enumerator,
    decider=decider
)

program = synthesizer.synthesize()

ppool = get_profiler_pool()
for p in ppool.keys():
    print(ppool[p])
    if recorder:
        recorder.record(("Profiler", str(ppool[p])))

print("# search space (skeletons): {}".format(len(enumerator._iters[:enumerator._iter_ptr+1])))
print("# search space (concrete programs): {}".format(
    sum([len(p) for p in enumerator._iters[:enumerator._iter_ptr+1]])
))

if program is None:
    logger.info('Exhausted.')
else:
    logger.info("Accepted: {}.".format(program))
    print("{}".format(str(program.to_jsonexp()).replace("'",'"')))

# loading skeleton list...
Profiler: trinity.interpreter.morpheus
|- counters
   |- cache_hit.gather: 2
   |- cache_hit.separate: 1
   |- interpreter.eval_gather: 3
   |- interpreter.eval_select: 1
   |- interpreter.eval_separate: 2
|- timers (cumulative)
   |- interpreter.equal_tb: 0.0
   |- interpreter.eval_gather: 0.02
   |- interpreter.eval_select: 0.0
   |- interpreter.eval_separate: 0.01
|- timers (sequential)

Profiler: trinity.enumerator.line_skeleton
|- counters
|- timers (cumulative)
   |- check_iterator: 0.0
   |- iterator.check_combination: 0.0
   |- iterator.construct_ast: 0.0
   |- iterator.next: 0.0
   |- next: 0.0
|- timers (sequential)

Profiler: trinity.decider.morpheus
|- counters
   |- analyze.equality: 1
   |- analyze.full_skeleton: 1
   |- analyze.partial_evaluation: 2
|- timers (cumulative)
   |- analyze.equality: 0.0
   |- analyze.full_skeleton: 0.0
   |- analyze.partial_evaluation: 0.04
   |- analyze.total: 0.04
|- timers (sequential)

Profiler: trinity.synthes

In [5]:
spec

In [6]:
spec.get_enum_production()

<bound method TrinitySpec.get_enum_production of <trinity.spec.spec.TrinitySpec object at 0x7fc0725cf7c0>>

In [7]:
spec.get_enum_production(spec.get_type("ColList"),['-99', '-1'])

EnumProduction(id=79, lhs=EnumType(ColList, domain=[['0'], ['1'], ['2'], ['3'], ['4'], ['5'], ['-99'], ['-1'], ['-2'], ['-3'], ['-4'], ['-5'], ['0', '1'], ['0', '2'], ['0', '3'], ['0', '4'], ['0', '5'], ['0', '-99'], ['0', '-1'], ['0', '-2'], ['0', '-3'], ['0', '-4'], ['0', '-5'], ['1', '2'], ['1', '3'], ['1', '4'], ['1', '5'], ['1', '-99'], ['1', '-1'], ['1', '-2'], ['1', '-3'], ['1', '-4'], ['1', '-5'], ['2', '3'], ['2', '4'], ['2', '5'], ['2', '-99'], ['2', '-1'], ['2', '-2'], ['2', '-3'], ['2', '-4'], ['2', '-5'], ['3', '4'], ['3', '5'], ['3', '-99'], ['3', '-1'], ['3', '-2'], ['3', '-3'], ['3', '-4'], ['3', '-5'], ['4', '5'], ['4', '-99'], ['4', '-1'], ['4', '-2'], ['4', '-3'], ['4', '-4'], ['4', '-5'], ['5', '-99'], ['5', '-1'], ['5', '-2'], ['5', '-3'], ['5', '-4'], ['5', '-5'], ['-99', '-1'], ['-99', '-2'], ['-99', '-3'], ['-99', '-4'], ['-99', '-5'], ['-1', '-2'], ['-1', '-3'], ['-1', '-4'], ['-1', '-5'], ['-2', '-3'], ['-2', '-4'], ['-2', '-5'], ['-3', '-4'], ['-3', '-5'], ['

In [8]:
spec.get_enum_production(spec.get_type("ColInt"),"2")

EnumProduction(id=11, lhs=EnumType(ColInt, domain=['0', '1', '2', '3', '4', '5']), choice=2)